In [1]:
import cv2
import numpy as np
import mediapipe as mp

In [2]:
TARGET_WIDTH = 640  # Try 480 / 640 / 720 if needed

def resize_frame(frame, target_width=640):
    h, w = frame.shape[:2]
    if w == target_width:
        return frame
    scale = target_width / w
    new_w = target_width
    new_h = int(h * scale)
    return cv2.resize(frame, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

In [3]:
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose(
    static_image_mode=False,
    model_complexity=1,
    enable_segmentation=False,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)

In [4]:
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b

    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-8)
    angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
    return angle

In [5]:
video_path = "pushup_vids\Day_46.mp4"
output_path = "pushup_vids\pushup_output_day_46.mp4"  # Final saved video

cap = cv2.VideoCapture(video_path)

# Read one frame to get size after resize
ret, test_frame = cap.read()
if not ret:
    raise RuntimeError("Could not read video")

test_frame = resize_frame(test_frame, TARGET_WIDTH)
h, w = test_frame.shape[:2]

fps = cap.get(cv2.CAP_PROP_FPS)
if fps == 0 or np.isnan(fps):
    fps = 30  # fallback

# VideoWriter setup
fourcc = cv2.VideoWriter_fourcc(*"mp4v")
out = cv2.VideoWriter(output_path, fourcc, fps, (w, h))

# Reset video to first frame
cap.set(cv2.CAP_PROP_POS_FRAMES, 0)

frame_idx = 0
pushup_count = 0
stage = None  # "up" or "down"

# Store pushup events
pushup_events = []

# Display control
DISPLAY_DURATION = 50
display_until_frame = -1

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_idx += 1

    # Resize
    frame = resize_frame(frame, TARGET_WIDTH)

    # MediaPipe
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    rgb.flags.writeable = False
    results = pose.process(rgb)
    rgb.flags.writeable = True

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        landmarks = results.pose_landmarks.landmark

        shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
        elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
        wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]

        angle = calculate_angle(shoulder, elbow, wrist)

        # Pushup logic (tune thresholds if needed)
        if angle > 160:
            stage = "up"
        if angle < 90 and stage == "up":
            stage = "down"
            pushup_count += 1

            pushup_events.append({
                "frame": frame_idx,
                "count": pushup_count
            })

            display_until_frame = frame_idx + DISPLAY_DURATION

    # Timed display
    if frame_idx <= display_until_frame:
        cv2.putText(
            frame,
            f"Pushups: {pushup_count}",
            (30, 60),
            cv2.FONT_HERSHEY_SIMPLEX,
            1.2,
            (0, 255, 0),
            3
        )

    # Show preview (optional)
    cv2.imshow("Pushup Counter", frame)

    # ✅ Write frame to output video
    out.write(frame)

    if cv2.waitKey(1) & 0xFF == 27:  # ESC
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print("Saved output video to:", output_path)
print("Pushup Events:")
for e in pushup_events:
    print(e)

<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:2: SyntaxWarning: invalid escape sequence '\p'
C:\Users\91800\AppData\Local\Temp\ipykernel_9396\274324807.py:1: SyntaxWarning: invalid escape sequence '\D'
  video_path = "pushup_vids\Day_46.mp4"
C:\Users\91800\AppData\Local\Temp\ipykernel_9396\274324807.py:2: SyntaxWarning: invalid escape sequence '\p'
  output_path = "pushup_vids\pushup_output_day_46.mp4"  # Final saved video


Saved output video to: pushup_vids\pushup_output_day_46.mp4
Pushup Events:
{'frame': 71, 'count': 1}
{'frame': 270, 'count': 2}
{'frame': 294, 'count': 3}
{'frame': 320, 'count': 4}
{'frame': 348, 'count': 5}
{'frame': 375, 'count': 6}
{'frame': 398, 'count': 7}
{'frame': 448, 'count': 8}
{'frame': 1686, 'count': 9}
{'frame': 1738, 'count': 10}
{'frame': 1820, 'count': 11}
{'frame': 2018, 'count': 12}


In [6]:
pushup_events_sorted = sorted(pushup_events, key=lambda x: x["frame"])

print("Sorted Pushup Events:")
for e in pushup_events_sorted:
    print(f"Pushup #{e['count']} at frame {e['frame']}")

Sorted Pushup Events:
Pushup #1 at frame 71
Pushup #2 at frame 270
Pushup #3 at frame 294
Pushup #4 at frame 320
Pushup #5 at frame 348
Pushup #6 at frame 375
Pushup #7 at frame 398
Pushup #8 at frame 448
Pushup #9 at frame 1686
Pushup #10 at frame 1738
Pushup #11 at frame 1820
Pushup #12 at frame 2018
